# 安装

In [1]:
# https://bibtexparser.readthedocs.io/en/master/install.html
!pip install bibtexparser

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 46 kB 2.2 MB/s eta 0:00:011
  Created wheel for bibtexparser: filename=bibtexparser-1.2.0-py3-none-any.whl size=36711 sha256=05ccede9d2a7f99e48dfdd6fc917b22e5d0b6d3d6a80341bf81444f323eb6ac4
  Stored in directory: /Users/zhang/Library/Caches/pip/wheels/9a/bf/37/abf100a7ad8c8bb49b300653a358ee67f195dfc471a1b56e4c
Successfully built bibtexparser
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and wil

# 解析

In [81]:
# BibTeX Format Description: http://www.bibtex.org/Format/
import bibtexparser
import pprint
import re

# 读取文件
with open('/Users/zhang/GoogleDrive/references/distributed-database/distributed-database.bib') as bibtex_file:
    bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)
    

## 生成Neo4j Cypher

In [ ]:
keys = set()
authors = {}
publishers = {}
for entry in bib_database.entries:
    keys.update(entry.keys())

def process_propety(value):
    result = re.sub('\\n', " \\\\n", value)
    result = re.sub('\{', " ", result)
#     result = re.sub('\\', " ", result)
    result = re.sub('\}', " ", result)
    result = re.sub("'", " ", result)
    return result

def parser_author(author):
    if " and " in author:
        result = [a.strip() for a in author.split(" and ")]
    elif "," in author:
        result = [a.strip() for a in author.split(",")]
    else:
        result = [author]
    
    if "et al" in result:
        result.remove("et al")
    
    return result

def parse_entry(entry):
    cql = []
    properties = {}
    ID = entry['ID']
    ENTRYTYPE = entry['ENTRYTYPE']
    author = None
    publisher = None
    
    if 'author' in entry:
        author = process_propety(entry['author'])
    if 'publisher' in entry:
        publisher = process_propety(entry['publisher'])
    for key in keys:
        if key not in entry:
            continue
        # need comment, abstract
        if key in ['ID', 'ENTRYTYPE', 'author', 'publisher', 'priority', 'file', 
                   'comment', 'abstract']:
            continue
        if key == 'keywords' and entry[key].startswith('prio'):
            continue
        
        properties[key] = process_propety(entry[key])
    
    if ID is None:
        return None
    cql.append("CREATE ({ID}:{ENTRYTYPE} {{ {properties} }})".format(
        ID=ID,ENTRYTYPE=ENTRYTYPE,                                                        
        properties=", ".join(["{name}: '{value}'".format(name=k, value=properties[k]) for k in properties])))
    
    if author is not None:
        this_authors = parser_author(author)
        for author in this_authors:
            if author in authors:
                author_id = authors[author]
            else:
                author_id = len(authors) + 1
                authors[author] = author_id
                cql.append("CREATE (author{author_id}:Author {{ name: '{author}' }})".format(
                    author_id=author_id,
                    author=author))
            cql.append("CREATE ({ID})-[:AUTHOR]->(author{author_id})".format(
                    ID=ID,
                    author_id=author_id))
    
    if publisher is not None:
        if publisher in publishers:
            publisher_id = publishers[publisher]
        else:
            publisher_id = len(publishers) + 1
            publishers[publisher] = publisher_id
            cql.append("CREATE (publisher{publisher_id}:Publisher {{ name: '{publisher}' }})".format(
                publisher_id=publisher_id,
                publisher=publisher))
        cql.append("CREATE ({ID})-[:PUBLISHER]->(publisher{publisher_id})".format(
            ID=ID, 
            publisher_id=publisher_id))
    
    return cql
    

In [83]:
for entry in bib_database.entries:
    cqls = parse_entry(entry)
    for cql in cqls:
        print(cql)

CREATE (Petrov2019:book { isbn: '9781492040347', groups: 'Overview', title: 'Database Internals', year: '2019' })
CREATE (author1:Author { name: 'Alex Petrov' })
CREATE (Petrov2019)-[:AUTHOR]->(author1)
CREATE (publisher1:Publisher { name: 'O Reilly Media, Inc.' })
CREATE (Petrov2019)-[:PUBLISHER]->(publisher1)
CREATE (Hellerstein2007a:article { doi: '10.1561/1900000002', pages: '141--259', volume: '1', journal: 'Foundations and Trends \textregistered  in Databases', groups: 'Overview', title: 'Architecture of a Database System', number: '2', year: '2007' })
CREATE (author2:Author { name: 'Joseph M. Hellerstein' })
CREATE (Hellerstein2007a)-[:AUTHOR]->(author2)
CREATE (author3:Author { name: 'Michael Stonebraker' })
CREATE (Hellerstein2007a)-[:AUTHOR]->(author3)
CREATE (author4:Author { name: 'James Hamilton' })
CREATE (Hellerstein2007a)-[:AUTHOR]->(author4)
CREATE (publisher2:Publisher { name: 'Now Publishers' })
CREATE (Hellerstein2007a)-[:PUBLISHER]->(publisher2)
CREATE (Abadi2012:a

## 删除Cypher

```
MATCH p=()-[r:AUTHOR]->() DELETE r
MATCH p=()-[r:PUBLISHER]->() DELETE r

MATCH (n:Author) DELETE n
MATCH (n:Publisher) DELETE n
MATCH (n:article) DELETE n
MATCH (n:book) DELETE n
MATCH (n:misc) DELETE n
MATCH (n:inproceedings) DELETE n
MATCH (n:techreport) DELETE n
MATCH (n:mastersthesis) DELETE n
```